In [ ]:
# pip install --upgrade transformers

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset

# Load CSV
df = pd.read_csv("/kaggle/input/chatbot-intents-300/chatbot_intents_300.csv")

df

2025-04-10 02:34:18.951348: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744252459.210073      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744252459.280472      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


,text,intent
0,Compare prices of iPhone 14,compare_prices
1,Can you compare iPhone 14 prices on Amazon and...,compare_prices
2,Show price comparison for MacBook Pro,compare_prices
3,Compare deals for the Nintendo Switch,compare_prices
4,Check the price difference between Galaxy S22 ...,compare_prices
...,...,...
298,What's your purpose?,other
299,Do you know jokes?,other
300,Can you sing?,other
301,Why is water wet?,other


In [ ]:
# Encode intent labels
label_encoder = LabelEncoder()
df["label"] = label_encoder.fit_transform(df["intent"])

# Split dataset
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

# Convert to HuggingFace Dataset format
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

# Tokenizer and model
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=len(label_encoder.classes_))

# Tokenize function
def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:


train_dataset = train_dataset.map(tokenize, batched=True)
val_dataset = val_dataset.map(tokenize, batched=True)

# Set format for PyTorch
train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])
val_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])



Map:   0%|          | 0/242 [00:00<?, ? examples/s]

Map:   0%|          | 0/61 [00:00<?, ? examples/s]

In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir="./intent_model",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=30,
    weight_decay=0.01,
    logging_dir="./logs",
    do_eval=True,
    logging_steps=10,  # change this if needed
    save_steps=500,
    report_to="none"  # <--- disables wandb, tensorboard, etc.

)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)



In [ ]:
import transformers
transformers.logging.set_verbosity_info()

print("Starting training...")
trainer.train()
print("Training finished!")

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: intent, text, __index_level_0__. If intent, text, __index_level_0__ are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 242
  Num Epochs = 30
  Instantaneous batch size per device = 4
  Training with DataParallel so batch size has been adjusted to: 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 930
  Number of trainable parameters = 66,955,779


Starting training...


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss
10,0.870400
20,0.344700
30,0.155500
40,0.031600
50,0.017600
60,0.008700
70,0.005400
80,0.003800
90,0.003300
100,0.002800


Saving model checkpoint to ./intent_model/checkpoint-500
Configuration saved in ./intent_model/checkpoint-500/config.json
Model weights saved in ./intent_model/checkpoint-500/model.safetensors
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
Saving model checkpoint to ./intent_model/checkpoint-930
Configuration saved in ./intent_model/checkpoint-930/config.json
Model weights saved in ./intent_model/checkpoint-930/model.safetensors


Training completed. Do not forget to share your model on huggingface.co/models =)




Training finished!


In [ ]:
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification

# ✅ Load tokenizer from the original base model
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

# ✅ Load your fine-tuned model weights from the checkpoint
model = DistilBertForSequenceClassification.from_pretrained("/kaggle/working/intent_model/checkpoint-930")

model.eval()

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/12040accade4e8a0f71eabdb258fecc2e7e948be/vocab.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/12040accade4e8a0f71eabdb258fecc2e7e948be/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/12040accade4e8a0f71eabdb258fecc2e7e948be/tokenizer_config.json
loading file chat_template.jinja from cache at None
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/12040accade4e8a0f71eabdb258fecc2e7e948be/config.json
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [ ]:
import numpy as np

label_encoder = LabelEncoder()
label_encoder.classes_ = np.array(['compare_prices', 'get_cheapest', 'other'])

In [ ]:
def predict_intent(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predicted_class_id = int(torch.argmax(logits, dim=1).item())
    predicted_label = label_encoder.inverse_transform([predicted_class_id])[0]
    return predicted_label

In [ ]:
import torch

test_sentences = [
    "Compare prices of iPhone 14",        # → compare_prices
    "What’s the cheapest MacBook?",       # → get_cheapest
    "What’s your name?",                  # → other
    "Check Amazon vs eBay prices",        # → compare_prices
    "Tell me a joke",                     # → other
    "Find me the best deal on Pixel 7"    # → get_cheapest
]

print("=== Intent Predictions ===")
for sentence in test_sentences:
    intent = predict_intent(sentence)
    print(f"Input: {sentence}\nPredicted Intent: {intent}\n")

=== Intent Predictions ===
Input: Compare prices of iPhone 14
Predicted Intent: compare_prices

Input: What’s the cheapest MacBook?
Predicted Intent: get_cheapest

Input: What’s your name?
Predicted Intent: other

Input: Check Amazon vs eBay prices
Predicted Intent: compare_prices

Input: Tell me a joke
Predicted Intent: other

Input: Find me the best deal on Pixel 7
Predicted Intent: get_cheapest

